## Tugas Klasifikasi Data Mining

Kelompok 8 :\
Satria Pinandita Abyatarsyah - 2000514\
Sabian Annaya Havid - 2005021\
Irfan Mochamad Esa - 2005568\
Muhammad Firmansyah - 2010021

In [1]:
import pandas as pd

#import library Scikit-learn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

#import pickle
import pickle

#import library untuk masing-masing algoritma
from sklearn.naive_bayes import GaussianNB

from sklearn import tree

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

#import metrik yang dibutuhkan untuk cross validation
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Read dataset train

In [2]:
df = pd.read_csv("G:\\Datasets\\klasifikasi\\train.csv", delimiter = "|")

# Tujuan

Memprediksi apakah customer melakukan transaksi fraud atau bukan

# Eksplorasi Data

Memeriksa info dari dataset untuk mengidentifikasi adanya data NULL

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1879 entries, 0 to 1878
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   trustLevel                 1879 non-null   int64  
 1   totalScanTimeInSeconds     1879 non-null   int64  
 2   grandTotal                 1879 non-null   float64
 3   lineItemVoids              1879 non-null   int64  
 4   scansWithoutRegistration   1879 non-null   int64  
 5   quantityModifications      1879 non-null   int64  
 6   scannedLineItemsPerSecond  1879 non-null   float64
 7   valuePerSecond             1879 non-null   float64
 8   lineItemVoidsPerPosition   1879 non-null   float64
 9   fraud                      1879 non-null   int64  
dtypes: float64(4), int64(6)
memory usage: 146.9 KB


,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


Setelah diperiksa ternyata tidak ada data null di dataset

Selain itu, kami menyimpulkan bahwa semua atribut yang ada relevan dengan tujuan prediksi. Oleh karena itu tidak ada atribut yang di drop.

==========================================================================

Lalu kami mencari keterkaitan customer yang melakukan fraud dengan trust level mereka

Trust level adalah tingkat kepercayaan food retailer dengan seorang customer. Trust level adalah attribut yang paling berkaitan dengan apakah customer akan melakukan fraud. Ini akan dibuktikan dengan hasil pencarian feature importance di tiap metode klasifikasi.

In [4]:
# Isi dan jumlah elemen dalam attribut fraud.
#0 = bukan fraud
#1 = fraud

df.fraud.value_counts().sort_index()

0    1775
1     104
Name: fraud, dtype: int64

In [5]:
# Isi dan jumlah elemen dalam attribut trust level
# Level trust dengan range 1 sampai 6

df.trustLevel.value_counts().sort_index()

1    332
2    347
3    318
4    289
5    302
6    291
Name: trustLevel, dtype: int64

Lalu kami menyaring dataset untuk hanya menampilkan data yang fraud saja untuk melihat trust level berapa saja yang melakukan fraud.

In [6]:
dfFraud = df[df.fraud==1]

Ternyata pelanggan yang melakukan fraud memiliki trust level yang rendah (1 dan 2 dari 6)

In [7]:
dfFraud.trustLevel.value_counts()

1    89
2    15
Name: trustLevel, dtype: int64

# Kelas Target

Menyimpan isi atribut fraud ke Y dan selain fraud ke X untuk nantinya displit.

In [8]:
Y = df.fraud 
X = df.drop("fraud",axis=1)

# Split Dataset

Membagi X dan Y menjadi train dan test untuk nantinya digunakan dalam proses klasifikasi.

In [9]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=123)

# Prediksi menggunakan berbagai algoritma

### Gaussian Naive Bayes

Memanggil algoritma Gaussian Naive Bayes

In [10]:
clf = GaussianNB()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
precv_gnb=acc
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Akurasi 0.8670212765957447
              precision    recall  f1-score   support

           0       1.00      0.86      0.93       361
           1       0.23      1.00      0.38        15

    accuracy                           0.87       376
   macro avg       0.62      0.93      0.65       376
weighted avg       0.97      0.87      0.90       376

[[311  50]
 [  0  15]]


Gaussian NB mengasumsi tidak ada keterkaitan antar atribut, maka feature importance tidak dapat dicari

Gaussian NB menggunakan cross validation 3-fold

In [11]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = GaussianNB()
scores = cross_val_score(clf, X=X, y=Y, cv=3, scoring = make_scorer(classification_report_with_accuracy_score))
score_gnb = scores
print(scores)

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       592
           1       0.23      0.94      0.38        35

    accuracy                           0.82       627
   macro avg       0.61      0.88      0.64       627
weighted avg       0.95      0.82      0.87       627

[[484 108]
 [  2  33]]
              precision    recall  f1-score   support

           0       1.00      0.83      0.91       592
           1       0.25      0.94      0.39        34

    accuracy                           0.84       626
   macro avg       0.62      0.89      0.65       626
weighted avg       0.96      0.84      0.88       626

[[494  98]
 [  2  32]]
              precision    recall  f1-score   support

           0       1.00      0.84      0.91       591
           1       0.27      1.00      0.42        35

    accuracy                           0.85       626
   macro avg       0.63      0.92      0.67       626
weighted avg       0.96     

### Decision tree

Memanggil algoritma Decision tree

In [12]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
precv_dt=acc
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Akurasi 0.976063829787234
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       361
           1       0.75      0.60      0.67        15

    accuracy                           0.98       376
   macro avg       0.87      0.80      0.83       376
weighted avg       0.97      0.98      0.97       376

[[358   3]
 [  6   9]]


Mencari attribut yang terpenting dalam mencari fraud

In [13]:
feature_importances = pd.DataFrame(clf.feature_importances_,index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)

print(feature_importances)

                           importance
scannedLineItemsPerSecond    0.242006
totalScanTimeInSeconds       0.213417
trustLevel                   0.196980
lineItemVoids                0.093555
lineItemVoidsPerPosition     0.088933
valuePerSecond               0.055376
scansWithoutRegistration     0.048808
grandTotal                   0.036163
quantityModifications        0.024763


Attribut diatas telah terurut dari yang paling penting dalam menentukan fraud atau tidaknya transaksi

Decision tree menggunakan cross validation 3-fold

In [14]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X=X, y=Y, cv=3, scoring = make_scorer(classification_report_with_accuracy_score))
score_dt=scores
print(scores)


              precision    recall  f1-score   support

           0       0.98      0.98      0.98       592
           1       0.72      0.66      0.69        35

    accuracy                           0.97       627
   macro avg       0.85      0.82      0.83       627
weighted avg       0.97      0.97      0.97       627

[[583   9]
 [ 12  23]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       592
           1       0.64      0.74      0.68        34

    accuracy                           0.96       626
   macro avg       0.81      0.86      0.83       626
weighted avg       0.97      0.96      0.96       626

[[578  14]
 [  9  25]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       591
           1       0.69      0.51      0.59        35

    accuracy                           0.96       626
   macro avg       0.83      0.75      0.78       626
weighted avg       0.96     

### Random forest

Memanggil algoritma Random forest

In [15]:
clf = RandomForestClassifier(n_estimators=50, random_state=123)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
precv_rf=acc
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))


Akurasi 0.9867021276595744
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       361
           1       1.00      0.67      0.80        15

    accuracy                           0.99       376
   macro avg       0.99      0.83      0.90       376
weighted avg       0.99      0.99      0.99       376

[[361   0]
 [  5  10]]


In [16]:
feature_importances = pd.DataFrame(clf.feature_importances_,index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)

print(feature_importances)


                           importance
trustLevel                   0.275139
totalScanTimeInSeconds       0.153191
scannedLineItemsPerSecond    0.149126
lineItemVoidsPerPosition     0.099001
valuePerSecond               0.087869
grandTotal                   0.081010
lineItemVoids                0.072523
scansWithoutRegistration     0.056474
quantityModifications        0.025666


Random forest menggunakan cross validation 3-fold

In [17]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = RandomForestClassifier(n_estimators=50, random_state=123)
scores = cross_val_score(clf, X=X, y=Y, cv=3, scoring = make_scorer(classification_report_with_accuracy_score))
score_rf=scores
print(scores)


              precision    recall  f1-score   support

           0       0.98      0.99      0.99       592
           1       0.83      0.69      0.75        35

    accuracy                           0.97       627
   macro avg       0.90      0.84      0.87       627
weighted avg       0.97      0.97      0.97       627

[[587   5]
 [ 11  24]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       592
           1       0.96      0.76      0.85        34

    accuracy                           0.99       626
   macro avg       0.97      0.88      0.92       626
weighted avg       0.99      0.99      0.98       626

[[591   1]
 [  8  26]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       591
           1       0.92      0.34      0.50        35

    accuracy                           0.96       626
   macro avg       0.94      0.67      0.74       626
weighted avg       0.96     

### XGBoost

In [18]:
clf = xgb.XGBClassifier(objective = "multi:softprob", num_class = 4, eval_metric = "mlogloss", max_depth = 24, gamma=0.1, subsample = 0.90,learning_rate=0.01, n_estimators = 10, nthread=-1, use_label_encoder=False)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)
precv_xgb=acc
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))

Akurasi 0.9920212765957447
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       361
           1       1.00      0.80      0.89        15

    accuracy                           0.99       376
   macro avg       1.00      0.90      0.94       376
weighted avg       0.99      0.99      0.99       376



In [19]:
feature_importances = pd.DataFrame(clf.feature_importances_,index = X_train.columns, columns=['importance']).sort_values('importance',ascending=False)

print(feature_importances)

                           importance
trustLevel                   0.414826
scannedLineItemsPerSecond    0.132589
totalScanTimeInSeconds       0.111765
lineItemVoidsPerPosition     0.103356
lineItemVoids                0.072808
valuePerSecond               0.067117
scansWithoutRegistration     0.045965
grandTotal                   0.026483
quantityModifications        0.025090


XGboost menggunakan cross validation 3-fold

In [20]:
def classification_report_with_accuracy_score(y_true, y_pred):    
    print(classification_report(y_true, y_pred)) 
    print(confusion_matrix(y_true, y_pred))
    return accuracy_score(y_true, y_pred) 
clf = xgb.XGBClassifier(objective = "multi:softprob", num_class = 3, eval_metric = "mlogloss", max_depth = 24, gamma=0.1, subsample = 0.90,learning_rate=0.01, n_estimators = 10, nthread=-1, use_label_encoder=False)
scores = cross_val_score(clf, X=X, y=Y, cv=3, scoring = make_scorer(classification_report_with_accuracy_score))
score_xgb=scores
print(scores)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       592
           1       0.80      0.69      0.74        35

    accuracy                           0.97       627
   macro avg       0.89      0.84      0.86       627
weighted avg       0.97      0.97      0.97       627

[[586   6]
 [ 11  24]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       592
           1       0.74      0.74      0.74        34

    accuracy                           0.97       626
   macro avg       0.86      0.86      0.86       626
weighted avg       0.97      0.97      0.97       626

[[583   9]
 [  9  25]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       591
           1       0.72      0.60      0.66        35

    accuracy                           0.96       626
   macro avg       0.85      0.79      0.82       626
weighted avg       0.96     

# Perbandingan tingkat akurasi tiap algoritma

Akurasi tiap algoritma

In [21]:
print(precv_gnb)
print(precv_dt)
print(precv_rf)
print(precv_xgb)

0.8670212765957447
0.976063829787234
0.9867021276595744
0.9920212765957447


Akurasi tiap algoritma setelah menggunakan cross validation 3 fold

In [22]:
print(score_gnb)
print(score_dt)
print(score_rf)
print(score_xgb)

[0.8245614  0.84025559 0.84664537]
[0.96650718 0.96325879 0.9600639 ]
[0.97448166 0.985623   0.96166134]
[0.97288676 0.97124601 0.96485623]


# Kesimpulan

Untuk permasalahan memprediksi transaksi fraud. Berdasarkan hasil akurasi diatas,  sebelum menggunakan cross validation tampaknya terlihat bahwa algoritma XGBoost mendapatkan akurasi tertinggi lalu akurasi terkecil didapat oleh algoritma Gaussian NB.

Setelah melakukan cross validation, Random forest memiliki tingkat akurasi setingkat atau sedikit melebihi XGBoost. Akurasi terkecil didapat oleh Gaussian NB.